Q1. Fine-tuning vs. RAG
In your own words:
1. What is fine-tuning in the context of LLMs? Describe what changes (or is learned) during fine-tuning.
   1. fine tuning is similiar with RAG pipeline, but it will focus on specific and accurated datasets and need more cpu computation for specific model pretraining assigning, depending on pretrained data source in concrete direction, more memory-used and cost
   2. RAG is more based on instantly vector embedding from external files input and RNN prompt 
2. What is Retrieval-Augmented Generation (RAG), and how is it different from fine-tuning in terms of where the knowledge lives and how it is updated?
   1. RAG relies on external data source not just inserted LLM system, like pdfs, webpages, docs, etc
   2. RAG uses external data as context and reference, but fine-tuning needs more specific model training, cost more computation
   3. In RAG, knowledge lives in context window and vector knowledge, updated from recursivly prompt and generated LLM answer. 
3. Give one realistic example where you would prefer RAG over fine-tuning, and one example where you would prefer fine-tuning over RAG. Explain your reasoning briefly for each.
   1. Prefer RAG when I needs more general summarization or researching suggestion or suggestion sys, etc 
   2. Prefer Fine-tuning when I needs specifc knowlege support, like teaching tutoring, finance master, etc. 

Q2. Document → Page → Block → Chunk
You are building a RAG system for complex PDFs (e.g., annual reports).
1. Describe the typical hierarchy Document → Page → Block → Chunk. What
does each level represent?
   1. Doc -> Whole report
   2. Page -> report is seperated into a series pages during parsing depending on page number 
   3. Block -> Page is seperated based on Paragraph or Sessions, etc, but structure more boundaried and well-componneted, includes ids, headings, pages_numbers, tables fields, etc. 
   4. Chunk -> Metric which seperate Block into fixed or semantic length of tokens, providing context to prompt of LLM 
1. List at least three possible block_type values (e.g., heading, paragraph, table,figure) and explain how you might treat each one differently during indexing. 
   1. heading: indexing as dictionary id to help query 
   2. paragraph: embeding into tokens and saved in vector db  
   3. table: indexing as nested dictionary ??? 

Q3. Chunking Strategy & Trade-offs
Suppose you need to index a 150-page technical report with text, tables and diagrams.
1. Compare fixed-size overlapping chunks (e.g., 500 tokens with 100 overlap) vs. structure-aware chunks (split by headings/sections).
   1. the overlapping chunks will support relationship between each chunk and avoid context loss
2. Give one example of how bad chunking could cause (a) missing important context or (b) hallucinated answers.
   1. if chunking size is too small, context window will be squeezed, which cause context data loss, the answer from LLM will show less nearest
3. How would you choose chunk size and overlap in this scenario, and what trade-offs are you making between recall, precision, cost and latency?
   1. larger chunk size, more tokens used and latency 
   2. larger overlap, more related between chunks and less data loss 
   3. if I choose, usually 500-800 chunk size and 150 overlap

Interview-Style Questions 1

Q4. LangChain RAG with LCEL
LangChain’s LCEL often uses patterns like:
```
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| parser
```
1. Explain what each part of this chain is doing: retriever ,format_docs , RunnablePassthrough , prompt , llm , parser .
   1. retriever: data source url
   2. format_docs: [pdf, txt, etc]
   3. RunnablePassthrough(): A passthrough function simply receives input and returns it unchanged, enabling pipelines to connect steps without modifying the data.
   4. prompt: embedding vector list from user
   5. llm: selected LLM model 
   6. parser: selected file parser from python lib 
2. Map these components to the standard RAG steps: retrieve → prepare context → prompt → generate → post-process.
   1. retriever -> retrieve 
   2. format_docs -> retrieve 
   3. llm -> prompt 
   4. parser -> prepare, retrieve 

Q5. Tool Calling & When Not to Use Agents
Answer the following:
1. Briefly describe the five main steps of tool calling / function calling in an LLM system (from tool definitions to final answer).
   1. Define tools, decide which to call, format input, call tool into model response, and return final answer to user.
2. What is the difference between a Chain and an Agent in LangChain or similar frameworks?
   1. Chain is a fixed sequence of steps, combined procedures in a streaming workflow
   2. and Agent decides dynamically which tool or step to use based on the input automatically, integrated with external command, tools, files, context and more choice for model training for token embedding 

Q6. Evaluating a RAG or Agentic System
You’ve built a RAG or Agentic RAG system and now need to evaluate it.
1. Name and describe three evaluation dimensions that are especially important for RAG (for example: retrieval relevance, answer relevance,faithfulness/grounding).
   1. retrieval relavance: measures how well the retriever selects the right chunks.
   2. answer relevance: measures how nearest the vector list of prompt and vector context from Vector DB 
2. For open-ended answers (free-text), why is LLM-as-Judge often more useful than simple string-overlap metrics like ROUGE or BLEU?
   1. ❌ 
3. Give one example of how you might combine automatic evaluation and  lightweight human evaluation in a realistic workflow.
   1. for embedding, I will use pretrained model to automate several metrics which will help me to select when to use contextual embedding and when to use token embedding, the metrics are answer relevance, multi-meaning words percentage, etc 

Section B – Coding / Practical

Interview-Style Questions 2
Q7. Chunkization Logic from Blocks to Chunks
You receive a parsed document as a list of blocks, for example:
```
blocks = [
 {"block_id": 1, "block_type": "heading", "text": "1. Introduction"},
 {"block_id": 2, "block_type": "paragraph", "text": "This report discusses
..."},
 {"block_id": 3, "block_type": "paragraph", "text": "Our main contributions ar
e ..."},
 {"block_id": 4, "block_type": "heading", "text": "2. Methods"},
 {"block_id": 5, "block_type": "paragraph", "text": "We collected data from
..."},
 # ...
]
```
Design a simple chunkization function in Python-like pseudo-code:
```
def make_chunks(blocks, max_tokens: int = 300, overlap_tokens: int = 50):
 """
 Returns a list of chunks, where each chunk is a dict like:
 {
 "chunk_id": int,
 "text": "...",
 "source_block_ids": [ ... ],
 "heading_path": ["1. Introduction", ...],
 }
 """
```
Requirements / hints:
1. Try to respect headings: when possible, avoid mixing content from different top-level sections in the same chunk.
2. Within a section, concatenate paragraph blocks until you are near max_tokens (you may approximate token count by word count).
3. Implement simple overlap between neighboring chunks (e.g., last overlap_tokens from previous chunk appear at the beginning of the next). You do not need to call a real tokenizer; you can approximate with len(text.split()) .
Focus on the logic and metadata, not exact code correctness.

In [ ]:
def make_chunks(blocks, max_tokens: int = 300, overlap_tokens: int = 50):
    """
    psudo-code
    
    start = 0
    dic = {}
    text_length = len([block.text for block in blocks].parse())
    cur = 0
    chunk_id = 0
    chunk_text = ""
    source_ids = []
    heading_path = []
    
    while start < text_length:
        end = start + max_tokens - overlap_tokens
        if end > text_length:
            end = text_length
        while cur < end:
            chunk_text += text_tokens[cur] + " "
            bid = find_block_id(cur, blocks)
            cur += 1
        dic = {
            "chunk_id": chunk_id, 
            "text": chunk_text.strip(), 
            "source_block_ids": source_ids, 
            "heading_path": heading_path
            }
        chunks.append(dic)
        chunk_id += 1
        overlap_start = max(start, end - overlap_tokens)
        start = overlap_start
        chunk_text = " ".join(text_tokens[start:end]) + " "
        source_ids = []
    return chunks
    """
    pass


Q8. Minimal RAG And Tool-Calling Chain
Minimal RAG pipeline using any vector store

Write Python for a minimal RAG flow using a high-level vector store API (no need to implement similarity search yourself):
1. Indexing step:
```
from some_embedding_lib import embed
from some_vector_store import VectorStore # e.g., FAISS/Chroma-like AP
I
docs = [
 {"doc_id": "doc-1", "text": "...."},
 {"doc_id": "doc-2", "text": "...."},
]
# 1) split docs into chunks (you may reuse your make_chunks() idea)
# 2) create embeddings for each chunk
# 3) add them to vector_store with metadata (doc_id, chunk_id, heading, e
tc.)
```
2. Query step (per question):
```
def answer_question(question: str) -> str:
 # a) embed the question
 # b) vector_store.similarity_search(question_embedding, k=5)
 # c) format retrieved chunks into a context string
 # d) build a prompt:
 # "Use ONLY the context below to answer the question...
 # Context:\n{context}\n\nQuestion: {question}"
 # e) call llm(prompt) and return the answer
```
Your answer should show the data flow clearly: documents → chunks →
embeddings → vector_store → retrieval → prompt → LLM.

In [ ]:
"""
Psedo code 

from some_embedding_lib import embed
from some_vector_store import VectorStore # e.g., FAISS/Chroma-like API
from make_chunks import make_chunks

docs = [
 {"doc_id": "doc-1", "text": "...."},
 {"doc_id": "doc-2", "text": "...."},
]

chunks = []
for doc in docs:
    doc_chunks = make_chunks(doc["text"])
    for c in doc_chunks:
        c.update({"doc_id": doc["doc_id"]})
        chunks.append(c)

vector_store = VectorStore()

for c in chunks:
    c_embedding = embed(c["text"])
    vector_store.add_vector(
        c_embedding, 
        metadata={
            "doc_id": c["doc_id"], 
            "chunk_id": c["chunk_id"], 
            "heading_path": c["heading_path"]
            })
"""

Simple LangChain tool-calling chain

Design a small LangChain-based tool-calling pipeline with one tool, e.g. a fake exchange-rate lookup:
```
from langchain_core.tools import tool
RATES = {("USD", "EUR"): 0.9, ("EUR", "USD"): 1.1}
@tool
def get_exchange_rate(base: str, quote: str) -> float:
 """Get the FX rate from base currency to quote currency.
 Use this when the user asks to convert or compare currencies."""
 return RATES[(base, quote)]
```
Tasks:
1. Show how you would bind this tool to a chat model (e.g., ChatOpenAI ) in
LangChain and create a tool-aware LLM.
2. Write the control flow for one full interaction:
```
user_message = "If I have 100 USD, how many EUR is that approximatel
y?"
# a) call llm_with_tools.invoke(user_message)
# b) inspect response.tool_calls
# c) execute the requested tool(s) with the given args
# d) send a follow-up message to the model with the tool results
# e) return the final natural-language answer to the user
```

You may omit import boilerplate, but the sequence of steps and the division between model and application code should be clear.